Наша первая задача - научиться делать Word Sense Disambiguation (WSD) при помощи семантических векторов.
Все необходимые библиотеки можно поставить при помощи команды

    pip install -U numpy scipy scikit-learn gensim rl_wsd_labeled
    
Скачайте word2vec модель по ссылке **TODO** - ссылка, сделать небольшую модель.

Загрузим word2vec модель при помощи библиотеки gensim (**TODO** - ссылка):

In [2]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load('../rlwsd/models/w2v.pkl')

Эта модель возвращает вектор по слову:

In [28]:
w2v_model['горшок']

array([ 0.14857584, -0.02300115,  0.21244895, ...,  0.19848056,
        0.07429154,  0.09368484], dtype=float32)

In [29]:
w2v_model['горшок'].shape

(1024,)

Она построена по лемматизированному корпусу, так что слова "горшка" в ней нет:

In [ ]:
w2v_model['горшка']

Загрузим размеченные контексты для слова "горшок".

Мы занимаемся WSD, это supervised задача, так что нам нужны размеченные данные:

In [44]:
import rl_wsd_labeled

senses, contexts = rl_wsd_labeled.get_contexts(
    rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', 'горшок'))

In [30]:
senses

{'1': 'Округлый глиняный сосуд для приготовления пищи (печной горшок)',
 '2': 'Расширяющийся кверху сосуд с отверстием в дне (цветочный горшок)',
 '3': 'Ночной горшок'}

Каждый контекст имеет левую часть, слово для которого мы разметили значение, и правую часть:

In [60]:
len(contexts), contexts[:3]

(406,
 [(('телевизор, - ковер, , - музыкальный центр, - стол, - аквариум, - 3 шкафа, - цветы в',
    ' горшках',
    ', - мелкие аксессуары.'),
   '2'),
  (('перевалить в больший горшок, стараясь не повредить корни. Я выращиваю базилик в',
    ' горшках',
    ' объемом 0,7-1 литр. Этого ему вполне хватает. Можно брать горшки большего объема'),
   '2'),
  (('удобрениями. Весна это лучшее время для preszhdane горшечные растения в больший',
    ' горшок',
    ' со свежей почвой. Большинство из нас хорошо знают chitatelite эту практику, так'),
   '2')])

In [33]:
left, _, right = contexts[1][0]
left, right

('перевалить в больший горшок, стараясь не повредить корни. Я выращиваю базилик в',
 ' объемом 0,7-1 литр. Этого ему вполне хватает. Можно брать горшки большего объема')

Видно что контексты не лемматизованы, но в нашей модели есть только лемматизованые слова. Исправим это - загрузим mystem:

In [62]:
from pymystem3 import Mystem
mystem = Mystem()

In [63]:
mystem.lemmatize(left)

['перевалить',
 ' ',
 'в',
 ' ',
 'больший',
 ' ',
 'горшок',
 ', ',
 'стараться',
 ' ',
 'не',
 ' ',
 'повреждать',
 ' ',
 'корень',
 '. ',
 'я',
 ' ',
 'выращивать',
 ' ',
 'базилик',
 ' ',
 'в',
 '\n']

Нас будут интересовать только слова, отбросим знаки препинания и разделители:

In [42]:
import re

def tokenize(s):
    return [t for t in mystem.lemmatize(s)
            if re.match('\w+$', t)]

tokenize(left)

['перевалить',
 'в',
 'больший',
 'горшок',
 'стараться',
 'не',
 'повреждать',
 'корень',
 'я',
 'выращивать',
 'базилик',
 'в']

Теперь построим векторное представление контекста: возьмем среднее векторов всех слов контекста:

In [41]:
import numpy as np

def context_repr(context):
    left, _, right = context
    words = tokenize(left) + tokenize(right)
    return np.mean([w2v_model[w] for w in words if w in w2v_model],
                    axis=0)

context_repr(contexts[1][0])

(1024,)

**Вопрос:** нужно ли использовать один и тот же лемматизатор для преобразования контекста и при построении word2vec модели?

Дальше подготовим данные для обучения:

In [75]:
from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import cross_val_score

word = 'горшок'
senses, contexts = rl_wsd_labeled.get_contexts(
    rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', word))
xs = [ctx for ctx, _ in contexts]
ys = np.array([int(s) - 1 for _, s in contexts])

Построим векторные представления всех контекстов:

In [76]:
xs_vec = np.array([context_repr(ctx) for ctx in xs])

И обучим модель ``NearestCentroid`` используя кросс-валидацию:

In [85]:
scores = cross_val_score(NearestCentroid(), X=xs_vec, y=ys, cv=5)
print('Accuracy: {:.2f} ± {:.2f}'.format(np.mean(scores), 2 * np.std(scores)))

Accuracy: 0.82 ± 0.07


Наша метрика тут - точность. Но сколько можно получить, предсказывая самое частотное значение?

In [82]:
labels, counts = np.unique(ys, return_counts=True)
print('Most frequent sense baseline: {:.2f}'.format(
        np.mean(ys == labels[counts.argmax()])))

Most frequent sense baseline: 0.57


Теперь можно обучить классификатор на всех примерах и задать ему свой контекст для дизамбигуации:

In [86]:
clf = NearestCentroid()
clf.fit(xs_vec, ys)

NearestCentroid(metric='euclidean', shrink_threshold=None)

In [89]:
def predict(ctx):
    pred, = clf.predict([context_repr(ctx)])
    return senses[str(pred + 1)]

predict(('он полил цветы в', 'горшках', 'и выключил свет'))

'Расширяющийся кверху сосуд с отверстием в дне (цветочный горшок)'

In [90]:
predict(('он сел на', 'горшок', ''))

'Ночной горшок'

Задания - выбирать по вкусу:
1. Можете ли вы заставить классификатор ошибиться?
2. На каких тренировочных примерах он ошибается и почему?
3. А какая точность для других слов?
4. Как точность зависит от количества слов для обучения?
5. ``NearestCentroid`` не умеет работать с косинусной мерой близости, но известно что она работает лучше чем евклидова - сделайте классификатор который использует её и сравните качество.
6. Попробуйте использовать другие классификаторы из ``scikit-learn``, например ближайшие соседи, логистическую регрессию, другие. Какой классификатор работает лучше всех?
7. В представлении контекста (``context_repr``) мы учитываем все слова. Но все ли слова одинаково важны? Можете ли вы изменить эту функцию так, чтобы она давала более качественное предстваление контекста?
8. Какой будет результат если не использовать семантические вектора, а непосредственно слова?
9. Насколько важно качество word2vec модели?